# Ruwaida Rashid

#### How can one mathematically establish a threshold of certainty above which a transaction can be classified as fraudulent by exploring distance from the last point of transaction, the ratio to median purchase price and if the transaction site was a repeat retailer?
###### How is fraudulent transaction defined in the dataset?
###### Does a relationship exist between these variables and chip and pin usage for verification purposes?
###### Is there a correlation between the mode of transaction and the likelihood of a fraudulent transaction?
I have an interest in data privacy and breaches and with the limited statistical and python knowledge I believe this credit card fraud analysis allows me to gain the basic technical knowledge needed to further understand the techniques involved in further complex data analysis. Following the factors associated with a credit card transaction, I would like to understand if a trend exists between the distance from last location of transaction with the same credit card and a fraudulent transaction. Given the complexity of real life scenarios, I am also interested to know if higher median transaction suggests a greater chance of fraudulent transaction going through.

In [2]:
import pandas as pd
dataset = pd.read_csv("../data/raw/card_transdata.csv")
dataset.head(n=20)

/var/folders/f0/nh99jdt16ddg2nsr3t4x3s7r0000gn/T/ipykernel_18328/2645359960.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("../data/raw/card_transdata.csv")


,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5,5.586408,13.261073,0.064768,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6,3.724019,0.956838,0.278465,1.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7,4.848247,0.320735,1.273050,1.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
8,0.876632,2.503609,1.516999,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
9,8.839047,2.970512,2.361683,1.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
